In [1]:
%%writefile preproccess.py

import logging
import os
import subprocess
import pandas as pd
import yaml
import datetime
import gc
import re

# File reading #

def read_config_file(filepath):
    with open(filepath,"r") as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)

def replacer(string, char):
    pattern=char+ '{2,}'
    string=re.sub(pattern,char,string)
    return string

def col_header_val(df,table_config):
#replace whitespaces in the columm and standarize col names
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]',_ ,regex=True)
    df.columns = list(map(lambda x: x.strip('='),list(df.columns)))
    df.columns = list (map(lambda x: replacer(x,'_'),list(df.columns)))
    expected_col = list (map(lambda x: x.lower(),table_config['columns']))
    expected_col.sort()
    df =df.reindex(sorted(df.columns),axis=1)

    if len(df.columns)== len(expected_col) and list(expected_col)==list(df.columns):
        print("column names and len validation passed")
        return 1
    else:
        print("column names and len validation failed")
        mismatched_col_file=list(set(df.columns).difference(expected_col))
        print("the following file columns are not in the yaml file",mismatched_col_file)
        missing_yaml_file=list(set(expected_col).difference(df.columns))
        print("the following yaml columns are not in the file uploader",missing_yaml_file)
        logging.info(f'df columns: (df.columns)')
        logging.info(f'expected columns: (expected_col)')
        return 0



Overwriting preproccess.py


In [2]:
import yaml
a_yaml_file = open("config.yaml")
config_data = yaml.load(a_yaml_file, Loader=yaml.FullLoader)

print(config_data["file_type"])

csv


In [3]:
config_data

{'file_type': 'csv',
 'dataset_name': 'nyt_comments',
 'file_name': 'nyt_comments',
 'table_name': 'nyt',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_loading_rows': 1,
 'columns': ['commentID',
  'status',
  'commentSequence',
  'userID',
  'userDisplayName',
  'userLocation',
  'userTitle',
  'commentBody',
  'createDate',
  'updateDate',
  'approveDate',
  'recommendations',
  'replyCount',
  'editorsSelection',
  'parentID',
  'parentUserDisplayName',
  'depth',
  'commentType',
  'trusted',
  'recommendedFlag',
  'permID',
  'isAnonymous',
  'articleID']}

In [4]:
import modin.pandas as pd

In [6]:
import modin.pandas as pd
import time 

start = time.time()
df = pd.read_csv('nyt_comments.csv',delimiter=',')
end = time.time()

print("Read csv with Modin: ",(end-start),"sec")
df.head()

To remove this warning, run the following python code before doing dataframe operations:

    import ray
    ray.init()

(pid=None) [2021-04-17 22:44:44,387 C 12652 5416] node_manager.cc:746:  Check failed: node_id != self_node_id_ Exiting because this node manager has mistakenly been marked dead by the monitor: GCS didn't receive heartbeats within timeout 30000 ms. This is likely since the machine or raylet became overloaded.
(pid=None) [2021-04-17 22:44:44,421 E 12652 5416] logging.cc:414: *** Aborted at 1618685084 (unix time) try "date -d @1618685084" if you are using GNU date ***
(pid=None) [2021-04-17 22:44:44,631 E 12652 5416] logging.cc:414:     @     0x7ffb88521881 raise
(pid=None) [2021-04-17 22:44:44,631 E 12652 5416] logging.cc:414:     @     0x7ffb88522851 abort
(pid=None) [2021-04-17 22:44:44,680 E 12652 5416] logging.cc:414:     @     0x7ff60a7a941b public: void __cdecl google::NullStreamFatal::`vbase destructor'(void) __ptr64
(pid=None) [2021-04-17 22:44:44,680 E 12652 5

RayTaskError: [36mray::deploy_ray_func()[39m (pid=8764, ip=192.168.0.143)
  File "python\ray\_raylet.pyx", line 490, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 491, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 1450, in ray._raylet.CoreWorker.store_task_outputs
  File "python\ray\_raylet.pyx", line 148, in ray._raylet.check_status
ray.exceptions.RaySystemError: System error: Unknown error

(pid=9596) Traceback (most recent call last):
(pid=9596)   File "python\ray\_raylet.pyx", line 490, in ray._raylet.execute_task
(pid=9596)   File "python\ray\_raylet.pyx", line 491, in ray._raylet.execute_task
(pid=9596)   File "python\ray\_raylet.pyx", line 1450, in ray._raylet.CoreWorker.store_task_outputs
(pid=9596)   File "python\ray\_raylet.pyx", line 148, in ray._raylet.check_status
(pid=9596) ray.exceptions.RaySystemError: System error: Unknown error
(pid=9596) 
(pid=9596) During handling of the above exception, another exception occurred:
(pid=9596) 
(pid=9596) Traceback (most recent call last):
(pid=9596)   File "python\ray\_raylet.pyx", line 550, in ray._raylet.task_execution_handler
(pid=9596)   File "python\ray\_raylet.pyx", line 417, in ray._raylet.execute_task
(pid=9596)   File "python\ray\_raylet.pyx", line 515, in ray._raylet.execute_task
(pid=9596)   File "C:\Users\M-ODE\anaconda3\lib\site-packages\ray\utils.py", line 96, in push_error_to_driver
(pid=9596)     worker.c

In [7]:
import pandas as pd
file_type=config_data['file_type']
source_file=config_data['file_name']+ f'.{file_type}'

start = time.time()
df_chunk=pd.read_csv(source_file,config_data['inbound_delimiter'],chunksize=100000)

chunk_list = []  # append each chunk df here 

# Each chunk is in df format
for chunk in df_chunk:  
    
    # Once the data filtering is done, append the chunk to list
    chunk_list.append(chunk)
    
# concat the list into dataframe 
df_final = pd.concat(chunk_list)
end = time.time()
print("Read csv with Panda: ",(end-start),"sec")

Read csv with Panda:  585.2432332038879 sec


In [8]:
df_final.head()

,commentID,status,commentSequence,userID,userDisplayName,userLocation,userTitle,commentBody,createDate,updateDate,...,editorsSelection,parentID,parentUserDisplayName,depth,commentType,trusted,recommendedFlag,permID,isAnonymous,articleID
0,104387472,approved,104387472,60215558,magicisnotreal,earth,NaN,Here is something I think is fraudulent that v...,2020-01-01 01:05:46,2020-01-01 08:13:39,...,False,NaN,NaN,1,comment,0,0,104387472,False,nyt://article/69a7090b-9f36-569e-b5ab-b0ba5bb3...
1,104387873,approved,104387873,65691034,JD,Elko,NaN,@magicisnotreal I have used my VA loan option...,2020-01-01 01:52:25,2020-01-01 20:55:19,...,False,104387472.0,magicisnotreal,2,userReply,0,0,104387873,False,nyt://article/69a7090b-9f36-569e-b5ab-b0ba5bb3...
2,104387976,approved,104387976,65110053,ebmem,"Memphis, TN",NaN,@magi\n\nWhy would someone take out a VA loan ...,2020-01-01 02:06:05,2020-01-01 20:55:35,...,False,104387472.0,magicisnotreal,2,userReply,0,0,104387976,False,nyt://article/69a7090b-9f36-569e-b5ab-b0ba5bb3...
3,104390628,approved,104390628,60215558,magicisnotreal,earth,NaN,@JD\nOut here in the Alabama of the PNW they w...,2020-01-01 14:38:50,2020-01-01 20:56:46,...,False,104387873.0,magicisnotreal,2,userReply,0,0,104390628,False,nyt://article/69a7090b-9f36-569e-b5ab-b0ba5bb3...
4,104391463,approved,104391463,65691034,JD,Elko,NaN,@magicisnotreal just a guess but I doubt that...,2020-01-01 16:23:14,2020-01-01 16:25:57,...,False,104390628.0,magicisnotreal,2,userReply,0,0,104391463,False,nyt://article/69a7090b-9f36-569e-b5ab-b0ba5bb3...


In [9]:
# using Dask 
import dask.dataframe as dd
file_type=config_data['file_type']
source_file=config_data['file_name']+ f'.{file_type}'

start = time.time()
df = dd.read_csv(source_file,delimiter=config_data['inbound_delimiter'])

chunk_list = []  # append each chunk df here 

end = time.time()
print("Read csv with Dask: ",(end-start),"sec")

Read csv with Dask:  149.55840682983398 sec


In [10]:
df.head()

,commentID,status,commentSequence,userID,userDisplayName,userLocation,userTitle,commentBody,createDate,updateDate,...,editorsSelection,parentID,parentUserDisplayName,depth,commentType,trusted,recommendedFlag,permID,isAnonymous,articleID
0,104387472,approved,104387472,60215558,magicisnotreal,earth,NaN,Here is something I think is fraudulent that v...,2020-01-01 01:05:46,2020-01-01 08:13:39,...,False,NaN,NaN,1,comment,0,0,104387472,False,nyt://article/69a7090b-9f36-569e-b5ab-b0ba5bb3...
1,104387873,approved,104387873,65691034,JD,Elko,NaN,@magicisnotreal I have used my VA loan option...,2020-01-01 01:52:25,2020-01-01 20:55:19,...,False,104387472.0,magicisnotreal,2,userReply,0,0,104387873,False,nyt://article/69a7090b-9f36-569e-b5ab-b0ba5bb3...
2,104387976,approved,104387976,65110053,ebmem,"Memphis, TN",NaN,@magi\n\nWhy would someone take out a VA loan ...,2020-01-01 02:06:05,2020-01-01 20:55:35,...,False,104387472.0,magicisnotreal,2,userReply,0,0,104387976,False,nyt://article/69a7090b-9f36-569e-b5ab-b0ba5bb3...
3,104390628,approved,104390628,60215558,magicisnotreal,earth,NaN,@JD\nOut here in the Alabama of the PNW they w...,2020-01-01 14:38:50,2020-01-01 20:56:46,...,False,104387873.0,magicisnotreal,2,userReply,0,0,104390628,False,nyt://article/69a7090b-9f36-569e-b5ab-b0ba5bb3...
4,104391463,approved,104391463,65691034,JD,Elko,NaN,@magicisnotreal just a guess but I doubt that...,2020-01-01 16:23:14,2020-01-01 16:25:57,...,False,104390628.0,magicisnotreal,2,userReply,0,0,104391463,False,nyt://article/69a7090b-9f36-569e-b5ab-b0ba5bb3...


## 1- Using Modin takes long time to run and after an hour of totaly consuming the allocated ram, it gave me error so i was not able to know the time needed to read the data

## 2- Using Pandas alone , consume the ram and causes the browser to slow down a lot, I tried waiting but it was not effiecent to read directly using pandas

## 3- Using Pandas with reading the csv file in chunks, i was able to read the data set in 585.2432332038879 sec 

## Using dask , I was able to read the data set in 149.55840682983398 sec

## So clearly dask is faster at reading the data set, but dask  doesn’t provide a wide range of data preprocessing functions like pandas, so in case of the need to use functions that is not supported, panads with chunks is prefered. otherwise dask offer parallel computing and will load the data faster.